In [ ]:
!pip install langchain --quiet
!pip install langchain-openai --quiet
!pip install langchain-community --quiet
!pip install streamlit --quiet
!pip install PyMuPDF --quiet
!pip install chromadb --quiet
!pip install pyngrok --quiet
!pip install langchain-community --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.1/571.1 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requiremen

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPEN_API_KEY')
os.environ['NGROK_AUTH_TOKEN'] = userdata.get('NGROK_AUTH_TOKEN')
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

In [ ]:
%%writefile app.py

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from operator import itemgetter
from langchain.retrievers import (
    MultiQueryRetriever,
    ContextualCompressionRetriever,
    )

from langchain.retrievers.document_compressors import (
    LLMChainExtractor,
    LLMChainFilter,
    CrossEncoderReranker
    )

from langchain_community.cross_encoders import HuggingFaceCrossEncoder

import streamlit as st
import tempfile
import os
import pandas as pd

# Customize initial app landing page
st.set_page_config(page_title="File QA Chatbot", page_icon="🤖")
st.title("Let's Chat With Your Files 🤖")

@st.cache_resource(ttl="1h")
def configure_retriever(uploaded_files):
    # Read documents
    docs = []
    temp_dir = tempfile.TemporaryDirectory()
    for file in uploaded_files:
        temp_filepath = os.path.join(temp_dir.name, file.name)
        with open(temp_filepath, "wb") as f:
            f.write(file.getvalue())
        loader = PyMuPDFLoader(temp_filepath)
        docs.extend(loader.load())

    # Split into document chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
    doc_chunks = text_splitter.split_documents(docs)

    # Create document embeddings and store in Vector DB
    embeddings_model = OpenAIEmbeddings()
    #vectordb = Chroma.from_documents(doc_chunks, embeddings_model)
    #vectordb = Chroma.from_documents(doc_chunks, embeddings_model, persist_directory=None, in_memory=True)
    persist_directory = "./chroma_db"
    vectordb = Chroma.from_documents(doc_chunks, embeddings_model, persist_directory=persist_directory)

    ##################################################################
    # The following combination of Reranker, Filter and Contextual Compression
    # Retriever takes much more time and almost gives similar output to that of
    # normal similarity based retriever.
    # Hence using a normal similarity based retriever only.

    ##################################################################


    # download an open-source reranker model - BAAI/bge-reranker-v2-m3
    #reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-large")
    #reranker_compressor = CrossEncoderReranker(model=reranker, top_n=3)

    # Define retriever object
    #chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

    #similarity_retriever_oai = vectordb.as_retriever(search_type="similarity",
     #                                         search_kwargs={"k": 5})
    #_filter = LLMChainFilter.from_llm(llm=chatgpt)
    #compressor_retriever_oai = ContextualCompressionRetriever(
    #base_compressor=_filter, base_retriever=similarity_retriever_oai)

    # Retriever 3 - Uses a Reranker model to rerank retrieval results from the previous retriever
    #final_retriever_oai = ContextualCompressionRetriever(
    #base_compressor=reranker_compressor, base_retriever=compressor_retriever_oai)

    retriever = vectordb.as_retriever()
    return retriever

# Callback to update UI as LLM streams responses
class StreamHandler(BaseCallbackHandler):
    def __init__(self, container, initial_text=""):
        self.container = container
        self.text = initial_text

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.text += token
        self.container.markdown(self.text)


# Create UI element to accept PDF uploads
uploaded_files = st.sidebar.file_uploader(
    label="Upload PDF files", type=["pdf"],
    accept_multiple_files=True
)
if not uploaded_files:
    st.info("Please upload PDF documents to continue.")
    st.stop()

# Create retriever object based on uploaded PDFs
retriever = configure_retriever(uploaded_files)

# Load a connection to ChatGPT LLM
chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.1, streaming=True)

# Update the prompt template to include chat history context
qa_template = """
Use the following conversation history:
{chat_history}

Use only the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know,
don't try to make up an answer. Keep the answer as concise as possible.

{context}

Question: {question}
"""
qa_prompt = ChatPromptTemplate.from_template(qa_template)

# This function formats retrieved documents before sending to LLM
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


# Create a QA RAG System Chain that now maps the user question, chat history, and retrieved context.
qa_rag_chain = (
    {
        "context": itemgetter("question")  # Retrieve context based on question
            | retriever
            | format_docs,
        "question": itemgetter("question"),  # User question
        "chat_history": itemgetter("chat_history")  # Conversation history
    }
    | qa_prompt  # Pass to prompt template which now expects chat_history, context, and question
    | chatgpt   # Send prompt to LLM for response
)


# Store conversation history in Streamlit session state
streamlit_msg_history = StreamlitChatMessageHistory(key="langchain_messages")

# Shows the first message when the app starts
if len(streamlit_msg_history.messages) == 0:
    streamlit_msg_history.add_ai_message("What would you like to ask to your documents?")

# Render current messages from StreamlitChatMessageHistory
for msg in streamlit_msg_history.messages:
    st.chat_message(msg.type).write(msg.content)

# Callback handler to display top 3 document sources used by the LLM
class PostMessageHandler(BaseCallbackHandler):
    def __init__(self, msg: st.write):
        BaseCallbackHandler.__init__(self)
        self.msg = msg
        self.sources = []

    def on_retriever_end(self, documents, *, run_id, parent_run_id, **kwargs):
        source_ids = []
        for d in documents:  # Retrieved documents from retriever based on user query
            metadata = {
                "source": d.metadata["source"],
                "page": d.metadata["page"],
                "content": d.page_content[:200]
            }
            idx = (metadata["source"], metadata["page"])
            if idx not in source_ids:  # Store unique source documents
                source_ids.append(idx)
                self.sources.append(metadata)

    def on_llm_end(self, response, *, run_id, parent_run_id, **kwargs):
        if len(self.sources):
            st.markdown("__Sources:__\n")
            st.dataframe(data=pd.DataFrame(self.sources[:3]), width=1000)  # Top 3 sources

# Helper function to compile chat history from message objects
def compile_chat_history(messages):
    history = ""
    for msg in messages:
        # Capitalize role for clarity
        role = msg.type.capitalize()
        history += f"{role}: {msg.content}\n"
    return history

# If user inputs a new prompt, display it and show the response
if user_prompt := st.chat_input():
    # Append human message to chat history and display it
    st.chat_message("human").write(user_prompt)
    streamlit_msg_history.add_user_message(user_prompt)

    # Compile chat history to pass to the chain
    chat_history_text = compile_chat_history(streamlit_msg_history.messages)

    # Use an AI message container for the response
    with st.chat_message("ai"):
        stream_handler = StreamHandler(st.empty())
        sources_container = st.write("")
        pm_handler = PostMessageHandler(sources_container)
        config = {"callbacks": [stream_handler, pm_handler]}

        # Pass both question and compiled chat history to the chain
        response = qa_rag_chain.invoke({"question": user_prompt, "chat_history": chat_history_text}, config)
        # Add AI's response to the message history
        streamlit_msg_history.add_ai_message(response)


Overwriting app.py


In [ ]:
!streamlit run app.py --server.port=8989 &>./logs.txt &

In [ ]:
from pyngrok import ngrok
import yaml

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken
# Get your authtoken from `ngrok_credentials.yml` file
# with open('ngrok_credentials.yml', 'r') as file:
#     NGROK_AUTH_TOKEN = yaml.safe_load(file)


In [ ]:
ngrok.set_auth_token(os.environ['NGROK_AUTH_TOKEN'] )

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://eb1c-35-245-21-126.ngrok-free.app
